In [1]:
import numpy as np
import tensorflow as tf

In [2]:
_04847_img = np.load('04847-image.npy')
_04799_img = np.load('04799-image.npy')
_04820_img = np.load('04820-image.npy')
_05675_img = np.load('05675-image.npy')
_05680_img = np.load('05680-image.npy')
_05710_img = np.load('05710-image.npy')

_04847_lbl = np.load('04847-label-onehot.npy')
_04799_lbl = np.load('04799-label-onehot.npy')
_04820_lbl = np.load('04820-label-onehot.npy')
_05675_lbl = np.load('05675-label-onehot.npy')
_05680_lbl = np.load('05680-label-onehot.npy')
_05710_lbl = np.load('05710-label-onehot.npy')

train_img = np.vstack((_04847_img, _04799_img, _04820_img, _05675_img, _05680_img))
train_lbl = np.vstack((_04847_lbl, _04799_lbl, _04820_lbl, _05675_lbl, _05680_lbl))

val_img = _05710_img
val_lbl = _05710_lbl

In [3]:
def next_batch(X, y, size):
    perm = np.random.permutation(X.shape[0])
    for i in np.arange(0, X.shape[0], size):
        yield (X[perm[i:i + size]], y[perm[i:i + size]])


def weight_variable(shape, name='W'):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)


def bias_variable(shape, name='b'):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


def loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train, X, Y, batch_size):
    c, l = 0, 0
    for batch_xs, batch_ys in next_batch(X, Y, batch_size):
        feed_dict = {x: batch_xs, y_: batch_ys, keep_prob: 1.0, phase_train: False}
        l += cross_entropy_count.eval(feed_dict=feed_dict)
        c += accuracy_count.eval(feed_dict=feed_dict)
    return l / Y.shape[0], c / Y.shape[0]

In [4]:
def conv_layer(input, channels_in, channels_out, name='conv', patch_x=15, patch_y=15):
    with tf.name_scope(name):
        # 15x15 patch, 8 channels, 32 output channels
        w = weight_variable([patch_x, patch_y, channels_in, channels_out], name='W')
        b = bias_variable([channels_out], name='B')

        conv = conv2d(input, w)
        bn = batch_norm(conv, channels_out, phase_train, name + '_bn')
        act = tf.nn.relu(bn)
        return max_pool_2x2(act)
    
def fc_layer(input, channels_in, channels_out, name='fc'):
    with tf.name_scope(name):
        w = weight_variable([channels_in, channels_out], name='W')
        b = bias_variable([channels_out], name='b')
        return tf.nn.relu(tf.matmul(input, w) + b)

In [5]:
def batch_norm(x, n_out, phase_train, name='bn'):
    """
    Batch normalization on convolutional maps.
    Ref.: http://stackoverflow.com/questions/33949786/how-could-i-use-batch-normalization-in-tensorflow
    Args:
        x:           Tensor, 4D BHWD input maps
        n_out:       integer, depth of input maps
        phase_train: boolean tf.Varialbe, true indicates training phase
        scope:       string, variable scope
    Return:
        normed:      batch-normalized maps
    """
    with tf.variable_scope(name):
        beta = tf.Variable(tf.constant(0.0, shape=[n_out]), name='beta', trainable=True)
        gamma = tf.Variable(tf.constant(1.0, shape=[n_out]), name='gamma', trainable=True)
        batch_mean, batch_var = tf.nn.moments(x, [0,1,2], name='moments')
        ema = tf.train.ExponentialMovingAverage(decay=0.6)

        def mean_var_with_update():
            ema_apply_op = ema.apply([batch_mean, batch_var])
            with tf.control_dependencies([ema_apply_op]):
                return tf.identity(batch_mean), tf.identity(batch_var)

        mean, var = tf.cond(phase_train,
                            mean_var_with_update,
                            lambda: (ema.average(batch_mean), ema.average(batch_var)))
        normed = tf.nn.batch_normalization(x, mean, var, beta, gamma, 1e-2)
    return normed

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 64, 64, 8], name='x')
x_image = tf.reshape(x, [-1, 64, 64, 8], name='x_reshaped')
y_ = tf.placeholder(tf.float32, shape=[None, 2], name='labels')
phase_train = tf.placeholder(tf.bool, name='phase_train')
keep_prob = tf.placeholder(tf.float32, name='keep_prob_fc1')

conv1 = conv_layer(x_image, 8, 32, 'conv1', 15, 15) # 15x15 patch, 8 channels, 32 output channels
conv2 = conv_layer(conv1, 32, 64, 'conv2', 15, 15) # 15x15 patch, 32 channels, 64 output channels
conv2_flat = tf.reshape(conv2, [-1, 16 * 16 * 64]) # flatten conv2

fc1 = fc_layer(conv2_flat, 16 * 16 * 64, 1024, 'fc1') # fc_layer fc1
fc2 = fc_layer(fc1, 1024, 512, name='fc2') # fc_layer fc2

# fc1_drop = tf.nn.dropout(fc1, keep_prob) # dropout on fc1

with tf.name_scope('fc3'):
    W_fc3 = weight_variable([512, 2])
    b_fc3 = bias_variable([2])
    y_conv = tf.matmul(fc2, W_fc3) + b_fc3

In [7]:
max_epochs = 20
max_steps = 1000
learning_rate = 1e-2
step, train_acc_arr, train_loss_arr, val_acc_arr, val_loss_arr = 0, [], [], [], []
with tf.name_scope('xent'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    cross_entropy_count = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_count = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))

In [8]:
sess = tf.InteractiveSession()
writer = tf.summary.FileWriter('/tmp/brain/2')
writer.add_graph(sess.graph)

sess.run(tf.global_variables_initializer())
for i in range(max_epochs):
    for batch_xs, batch_ys in next_batch(train_img, train_lbl, 10):
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: 0.5, phase_train: True})
        step += 1
    
        if step % 100 == 0:
            feed_dict = {x: batch_xs, y_: batch_ys, keep_prob: 1.0, phase_train: False}
            train_accuracy = accuracy.eval(feed_dict=feed_dict)
            train_loss = cross_entropy.eval(feed_dict=feed_dict)
            val_loss, val_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train,
                                                   val_img, val_lbl, 100)
            train_acc_arr.append(train_accuracy), train_loss_arr.append(train_loss)
            #val_acc_arr.append(val_accuracy), val_loss_arr.append(val_loss)

            print "step %d, train accuracy %.4f, train loss %.5f, val accuracy %g, val loss %g" % (
                step, train_accuracy, train_loss, val_accuracy, val_loss)

step 100, train accuracy 0.2000, train loss 1.95381, val accuracy 0.496875, val loss 0.839133
step 200, train accuracy 0.5000, train loss 1.21065, val accuracy 0.5125, val loss 1.14364
step 300, train accuracy 0.5000, train loss 0.69910, val accuracy 0.498437, val loss 0.696918
step 400, train accuracy 0.8000, train loss 0.64269, val accuracy 0.503125, val loss 0.729011
step 500, train accuracy 0.9000, train loss 0.68013, val accuracy 0.5125, val loss 0.696295
step 600, train accuracy 0.5000, train loss 0.70038, val accuracy 0.496875, val loss 0.702727
step 700, train accuracy 0.4000, train loss 0.70338, val accuracy 0.5, val loss 0.701474
step 800, train accuracy 0.8000, train loss 0.66438, val accuracy 0.5, val loss 0.696141
step 900, train accuracy 0.6000, train loss 0.69806, val accuracy 0.504687, val loss 0.69586
step 1000, train accuracy 0.7000, train loss 0.67964, val accuracy 0.489063, val loss 0.696981
step 1100, train accuracy 0.9000, train loss 0.63576, val accuracy 0.5, val

In [9]:
test_loss, test_accuracy = loss_accuracy(cross_entropy_count, accuracy_count, x, y_, keep_prob, phase_train,
                                         val_img, val_lbl, 100)
print 'Val Accuracy: %g, Val Loss: %g' % (test_accuracy * 100, test_loss)

Val Accuracy: 50, Val Loss: 0.69318
